# Lab2

### 1. 
- Для кожної із адміністративних одиниць України завантажити тестові
структуровані файли, що містять значення VHI-індексу. Ця процедура має бути
автоматизована, параметром процедури має бути індекс (номер) області.
 - При зберіганні файлу до його імені потрібно додати дату та час завантаження.

- Передбачити повторні запуски скрипту, довантаження нових даних та колізію
даних;

In [2]:
import urllib.request
from datetime import datetime
import os
import glob
import pandas as pd
import re

In [3]:
directory="./Data"

for i in range(1,29):
    url=f'https://www.star.nesdis.noaa.gov/smcd/emb/vci/VH/get_TS_admin.php?country=UKR&provinceID={i}&year1=1981&year2=2024&type=Mean'.format()
    
    name="NOAA_ID"+str(i)+"*"
    if glob.glob(os.path.join(directory, name)):
        print("The file already exists: " + str(i))
        continue
    
    now = datetime.now().strftime("%d%m%Y%H%M%S")
    
    wp = urllib.request.urlopen(url)
    text = wp.read()

    filename = os.path.join(directory, 'NOAA_ID'+str(i)+'_'+now+'.csv')
    out = open(filename,'wb')
    out.write(text)
 
    out.close()
    print(f"Downloaded file for {i} region")

The file already exists: 1
The file already exists: 2
The file already exists: 3
The file already exists: 4
The file already exists: 5
The file already exists: 6
The file already exists: 7
The file already exists: 8
The file already exists: 9
The file already exists: 10
The file already exists: 11
The file already exists: 12
The file already exists: 13
The file already exists: 14
The file already exists: 15
The file already exists: 16
The file already exists: 17
The file already exists: 18
The file already exists: 19
The file already exists: 20
The file already exists: 21
The file already exists: 22
The file already exists: 23
The file already exists: 24
The file already exists: 25
The file already exists: 26
The file already exists: 27
The file already exists: 28


### 2. Зчитати завантажені текстові файли у фрейм

In [4]:
def reading_text_into_the_frame(directory):
    
    headers = ['Year', 'Week', 'SMN', 'SMT', 'VCI', 'TCI', 'VHI', 'empty']
    all_df = []
    fnames = os.listdir(directory)
   
    for fname in fnames:
        
        if fname.startswith("NOAA") and fname.endswith(".csv"):
            
            file_path = os.path.join(directory, fname)
            df = pd.read_csv(file_path, header=1, names=headers)
            df =df.drop(columns=['empty']).dropna()
            df = df.drop(df.loc[df['VHI'] == -1].index)
            
            i = int(re.search(r'\d+', fname).group())
            df['area'] = i
            
            df.loc[0, 'Year'] = df.loc[0,'Year'][9:]
            
            df['Year'] = df['Year'].astype(int)
            
            all_df.append(df)
            
    final_df = pd.concat(all_df, ignore_index=True)
    
    #final_df = final_df.drop(columns=['empty'])
    print(final_df)
    
    print(final_df.dtypes)
    
    return(final_df)


final_df = reading_text_into_the_frame(directory)

       Year  Week    SMN     SMT    VCI    TCI    VHI  area
0      1982   1.0  0.059  258.24  51.11  48.78  49.95    10
1      1982   2.0  0.063  261.53  55.89  38.20  47.04    10
2      1982   3.0  0.063  263.45  57.30  32.69  44.99    10
3      1982   4.0  0.061  265.10  53.96  28.62  41.29    10
4      1982   5.0  0.058  266.42  46.87  28.57  37.72    10
...     ...   ...    ...     ...    ...    ...    ...   ...
60923  2024  38.0  0.164  300.27  26.53  11.33  18.95     9
60924  2024  39.0  0.158  298.55  28.21  10.60  19.43     9
60925  2024  40.0  0.153  296.53  30.42  11.37  20.91     9
60926  2024  41.0  0.147  294.02  32.65  12.27  22.49     9
60927  2024  42.0  0.140  291.36  34.00  12.81  23.45     9

[60928 rows x 8 columns]
Year      int64
Week    float64
SMN     float64
SMT     float64
VCI     float64
TCI     float64
VHI     float64
area      int64
dtype: object


### 3. Реалізувати окрему процедуру, яка змінить індекси областей, за англійською абеткою на наступні, за українською:

In [5]:

def change_index():
    
    
    index_ch = {1:22, 2:24, 3:23, 4:25, 5:3, 6:4, 7:8, 8:19, 9:20,
                10:21, 11:9, 12:9, 13:10,  14:11, 15:12, 16:13, 17:14,
                18:15 ,19:16, 20:26, 21:17 ,22:18 ,23:6 ,24:1 ,25:2 ,26:7 ,27:5}
    final_df["area"] = final_df["area"].replace(index_ch)
    
    print(final_df)

change_index()


       Year  Week    SMN     SMT    VCI    TCI    VHI  area
0      1982   1.0  0.059  258.24  51.11  48.78  49.95    21
1      1982   2.0  0.063  261.53  55.89  38.20  47.04    21
2      1982   3.0  0.063  263.45  57.30  32.69  44.99    21
3      1982   4.0  0.061  265.10  53.96  28.62  41.29    21
4      1982   5.0  0.058  266.42  46.87  28.57  37.72    21
...     ...   ...    ...     ...    ...    ...    ...   ...
60923  2024  38.0  0.164  300.27  26.53  11.33  18.95    20
60924  2024  39.0  0.158  298.55  28.21  10.60  19.43    20
60925  2024  40.0  0.153  296.53  30.42  11.37  20.91    20
60926  2024  41.0  0.147  294.02  32.65  12.27  22.49    20
60927  2024  42.0  0.140  291.36  34.00  12.81  23.45    20

[60928 rows x 8 columns]


### 4.1 Реалізувати ряд VHI для області за вказаний рік;
    

In [6]:
def vhi_region(index, year):
    return final_df[(final_df['area'] == index) & (final_df["Year"] == year)][['Year','area','VHI']]

print(vhi_region(5, 1989))

       Year  area    VHI
39521  1989     5  33.32
39522  1989     5  33.05
39523  1989     5  32.65
39524  1989     5  32.23
39525  1989     5  31.80
39526  1989     5  31.74
39527  1989     5  31.22
39528  1989     5  31.46
39529  1989     5  31.49
39530  1989     5  32.73
39531  1989     5  31.95
39532  1989     5  35.84
39533  1989     5  38.89
39534  1989     5  40.15
39535  1989     5  41.19
39536  1989     5  42.73
39537  1989     5  45.27
39538  1989     5  48.47
39539  1989     5  52.03
39540  1989     5  56.61
39541  1989     5  55.63
39542  1989     5  53.82
39543  1989     5  52.72
39544  1989     5  50.35
39545  1989     5  49.60
39546  1989     5  49.08
39547  1989     5  47.55
39548  1989     5  46.49
39549  1989     5  45.26
39550  1989     5  46.61
39551  1989     5  45.93
39552  1989     5  44.90
39553  1989     5  46.33
39554  1989     5  47.59
39555  1989     5  49.14
39556  1989     5  48.67
39557  1989     5  46.55
39558  1989     5  47.44
39559  1989     5  47.00


### 4.2 Реалізувати пошук екстремумів (min та max) для вказаних областей та років, середнього, медіани;

In [7]:
def extremes(year, regions):
    
    data = final_df[(final_df['Year'].between(year[0],year[1]))&
                    (final_df['area'].isin(regions))]
    
    min_v = data['VHI'].min()
    max_v = data['VHI'].max()
    mean_v = data['VHI'].mean()
    median_v = data['VHI'].median()
    
    return("Extremes: min-" + str(min_v) +", max-" + str(max_v) +", mean-" + str(mean_v) +", min-" + str(median_v))


print(extremes([ 1984, 1990], [11, 14, 16]))
    
    

Extremes: min-12.45, max-86.6, mean-43.05583569405099, min-41.57


### 4.3 Реалізувати ряд VHI за вказаний діапазон років для вказаних областей;

In [8]:
def sf_range(year, region):
    
    data = final_df[(final_df['Year'].between(year[0],year[1]))&
                    (final_df['area'].isin(region))]
    
    f_data = data.sort_values(by='area')
    
    return f_data[['Year', 'area', 'VHI']]

print(sf_range([1987, 1995], [4, 11]))


       Year  area    VHI
52919  1995     4  19.36
52918  1995     4  23.87
52917  1995     4  27.10
52916  1995     4  31.05
52915  1995     4  35.81
...     ...   ...    ...
8973   1987    11  42.26
8972   1987    11  45.26
8971   1987    11  47.96
8970   1987    11  49.05
8953   1987    11  58.60

[898 rows x 3 columns]


### 4.4 Виявити роки, протягом яких екстремальні посухи торкнулися більше вказаного відсотка областей по Україні 


In [9]:
def droughts(percent):
    
    count = int((percent*25)/100)
    print(count)
    drought = final_df[final_df['VHI']<15]
    count_of_region = drought.groupby(by='Year')['area'].count()
    years_d = count_of_region[count_of_region > count].index
    
    drought_avg = drought[drought['Year'].isin(years_d)].groupby(['Year', 'area' ],as_index=False)['VHI'].mean()
    
    #return drought[drought['Year'].isin(years_d)][['Year', 'VHI', 'area']]
    return drought_avg[['Year', 'VHI', 'area']]

print(droughts(20))
    

5
    Year        VHI  area
0   1993   8.872000     4
1   1993  14.080000    11
2   2000  12.213333     1
3   2000  10.436667     9
4   2000  11.460000    19
5   2000  12.415000    22
6   2000  10.828571    26
7   2007  12.641818     7
8   2007   7.895000    13
9   2007   8.114444    14
10  2007  13.289091    20
11  2007  14.008333    25
